In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

You can read the paper for more detail. <BR>
Reference: https://ieeexplore.ieee.org/abstract/document/5197422

In [2]:
class Matrix_Factorization:
    def __init__(self, alpha = 0.00001, iterations = 50, num_of_latent = 200, lam = 0.0005):
        """
            Some initializations, if neccesary
            
            attributes: 
                        alpha: Learning Rate, default 0.01
                        num_iter: Number of Iterations to update coefficient with training data
                        num_of_latent: Number of latent factor.
                        lam: Regularization constant
                        
            
            TODO: 1. Initialize all variables needed.
        """
        
        self.alpha = alpha
        self.iterations = iterations
        self.num_of_latent = num_of_latent
        self.lam = lam
        
    def fit(self, train):
        """
            Train: list of tuples with (User, Movie, Rating)
            num_user: Number of unique user.
            num_movie: Number of unique movie
            
            TODO: 2. Initialize num_user and num_movie
                  3. Save the training set.
                  4. Initialize P and Q matrix, with normal distribution with mean = 0. 
                  Hint: Think about what P and Q represent, what they should do.Think about the shape too. 
            
        
        """
        num_user = int(max([t[0] for t in train])) + 1
        num_movie = int(max([t[1] for t in train])) + 1
        self.train = train
        
        self.train = train

        
        self.P = np.random.normal(scale=0.1, size=(num_user, self.num_of_latent))
        self.Q = np.random.normal(scale=0.1, size=(num_movie, self.num_of_latent))
        
        rmse_lst = []
        
        """
            TODO: 5: Calculate the error, using P and Q matrix. 
                  6: We need to check if the absolute value error is less than some constant. Store the previous Q and P for adaptive learning rate.
                      If it is less than that constant then we update P and Q matrix. 
                      (When update, update the P and Q at the same time. Think about why it is important.)
                      Otherwise use the error to update the Q and P matrix.
                      
                  7: For each entry update temp_mse, and append the Current iteration RMSE to rmse_lst.
                  
        """
 
        for f in range(self.iterations):
            ### Random Shuffle. Why is this called?
            np.random.shuffle(self.train)
            
            temp_mse = 0

            previous_Q = self.Q.copy()
            previous_P = self.P.copy()
            
            Count = 0
   
            for tup in self.train:
                u,i,rating = tup
                error = rating - np.dot(self.P[u], self.Q[i])
                
                if abs(error) > 20:
                    continue

                Count += 1
                temp_mse += error ** 2
                
                pu = self.P[u].copy()
                qi = self.Q[i].copy()
                
                self.P[u] = pu + self.alpha * (error * qi - self.lam * pu)
                self.Q[i] = qi + self.alpha * (error * pu - self.lam * qi)
                
                #### Don't Modify this code, helpful for converge.
                if (np.isinf(self.P[u]).any() or np.isnan(self.P[u]).any() or 
                    np.isinf(self.Q[i]).any() or np.isnan(self.Q[i]).any()):
                    self.P[u] = pu
                    self.Q[i] = qi
            
            if Count > 0:
                rmse = np.sqrt(temp_mse / Count)
            else:
                rmse = float('inf')
            rmse_lst.append(rmse)
            
            
            """
                TODO: 8: Implement the adaptive learning rate.
                         If current rmse is less than previous iteration, let's increase by a factor range from 1 - 1.5
                         Otherwise we decrease by a factor range from 0.5 - 1
                      9: If the current rmse is greater than previous iteration.
                         Check the relative error, (previous - current)/ previous.
                         If it is greater than 0.1, we restore the previous Q and P. (Try without it. Think about why we need this.)
            """
            if f > 0:
                previous_rmse = rmse_lst[f-1]
                current_rmse = rmse_lst[f]
                if current_rmse < previous_rmse:
                    self.alpha *= 1.2
                else:
                    self.alpha *= 0.8
                    if previous_rmse > 0 and ((previous_rmse - current_rmse) / previous_rmse > 0.1):
                        self.P = previous_P
                        self.Q = previous_Q
                
        self.rmse = rmse_lst
        
    def ind_predict(self, tup):
        """
            tup: One single entry, (user, movie)
            
            TODO: 10: Use P and Q to make prediction on single entry.
            
        """
        u,i = int(tup[0]), int(tup[1])
        return np.dot(self.P[u, :], self.Q[i, :])
    
    def predict(self, X):
        """
            X: list of entries
            
            TODO: 11: Use ind_predict we create to make predicitons.
        """
        return [self.ind_predict(tup) for tup in X]

In [3]:
ratings = pd.read_csv('./ml-latest-small/ratings.csv')
ratings = ratings.sort_values('timestamp').reset_index(drop = True).astype(int)
a = np.sort(ratings.movieId.unique())
index_dict = {}
for i in range(len(a)):
    index_dict[a[i]]  = i
ratings.movieId = ratings.movieId.apply(lambda x: index_dict[x]) 

grouped_userId = list(ratings.groupby(['userId']).apply(lambda x: x[['movieId','rating']].values))
grouped_movieId = list(ratings.groupby(['movieId']).apply(lambda x: x[['userId','rating']].values))

/var/folders/8y/fh_657xx0p33nz_x8d0yxd300000gn/T/ipykernel_22560/1764614809.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_userId = list(ratings.groupby(['userId']).apply(lambda x: x[['movieId','rating']].values))
/var/folders/8y/fh_657xx0p33nz_x8d0yxd300000gn/T/ipykernel_22560/1764614809.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_movieId = list(ratings.groupby(['movieId']).ap

In [4]:
np.random.seed(0)
train = set()
test = set()

for i in range(len(grouped_movieId)):
    temp_len = len(grouped_movieId[i])
    for j in range(temp_len):
        if j < temp_len * 0.8:
            train.add((grouped_movieId[i][j][0]-1,i - 1, grouped_movieId[i][j][1]))
        else:
            test.add((grouped_movieId[i][j][0] -1, i - 1,grouped_movieId[i][j][1]))
train = list(train)
test = list(test)

#### Try to understand what is done for data cleaning. 

#### If you are doing things correctly (with tuning), you will get something lower than 1.1 test RMSE.

In [ ]:
%%time
clf = Matrix_Factorization(iterations =70, num_of_latent = 100, lam = 0.1)
clf.fit(train)

In [ ]:
error = 0
pred = clf.predict(test)
for i in range(len(test)):
    error += (test[i][2] - pred[i])**2

In [ ]:
(error/ len(test))** (0.5)

np.float64(1.0696613592540112)

In [5]:
class Matrix_Factorization_with_bias:
    def __init__(self, alpha = 0.00001, iterations = 50, num_of_latent = 200, lam = 0.01):
        
        """
            Some initializations, if neccesary
            
            attributes: 
                        alpha: Learning Rate, default 0.01
                        num_iter: Number of Iterations to update coefficient with training data
                        num_of_latent: Number of latent factor.
                        lam: Regularization constant
 
            
            TODO: 1. Initialize all variables needed.
        """
            
        self.alpha = alpha
        self.iterations = iterations
        self.num_of_latent = num_of_latent
        self.lam = lam


        
    def fit(self, train):
        """
            Train: list of tuples with (User, Movie, Rating)
            num_user: Number of unique user.
            num_movie: Number of unique movie
            
            TODO: 2. Initialize num_user and num_movie.
                  3. Save the training set.
                  4. Initialize bu , bi and b. b is the global mean of the rating.
                  5. Initialize P and Q matrix. 
                  Hint: Think about what P and Q represent, what they should do.Think about the shape too. 
                  
            
        
        """
        
        num_user = int(max([t[0] for t in train])) + 1
        num_movie = int(max([t[1] for t in train])) + 1
        self.train = train
        
        self.P = np.random.normal(0, 0.1, (num_user, self.num_of_latent))
        self.Q = np.random.normal(0, 0.1, (num_movie, self.num_of_latent))
        
        self.bu = np.zeros(num_user)
        self.bi = np.zeros(num_movie)
        self.b = np.mean([t[2] for t in train])
        
        rmse_lst = []
        
        """
            TODO: 5: Calculate the error, using P , Q , bu , bi and b. 
                  6: Update the P , Q , bu , bi and b with error you calculate. 
                    (Think about why we don't need to check the absolute of error)
                  7: For each entry update temp_mse, and append the Current iteration RMSE to rmse_lst.
                  
        """


        for f in range(self.iterations):
            
            np.random.shuffle(self.train)
            
            temp_mse = 0
            previous_Q = self.Q.copy()
            previous_P = self.P.copy()
            previous_bu = self.bu.copy()
            previous_bi = self.bi.copy()
            previous_b = self.b
            
            Count = 0
            for tup in self.train:
                u,i,rating = tup
                
                error = rating - (self.b + self.bu[u] + self.bi[i] + np.dot(self.P[u], self. Q[i]))         

                Count += 1   
                temp_mse += error ** 2
                
                self.bu[u] += self.alpha * (error - self.lam * self.bu[u])
                self.bi[i] += self.alpha * (error - self.lam * self.bi[i])
                self.b += self.alpha * error
                
                temp_P = self.P[u].copy()
                
                self.P[u] += self.alpha * (error * self.Q[i] - self.lam * self.P[u])
                self.Q[i] += self.alpha * (error * temp_P - self.lam * self.Q[i])
                
            
            if Count > 0:
                rmse = np.sqrt(temp_mse / Count)
            else:
                rmse = float('inf')
            rmse_lst.append(rmse)
                
            
            
            """
                TODO: 8: Implement the adaptive learning rate.
                         If current rmse is less than previous iteration, let's increase by a factor range from 1 - 1.5
                         Otherwise we decrease by a factor range from 0.5 - 1
                      9: If the current rmse is greater than previous iteration.
                         Check the relative error, (previous - current)/ previous.
                         If it is greater than 0.1, we restore the previous Q and P. (Try without it. Think about why we need this.)
            """
                
            
            if f > 0:
                previous_rmse = rmse_lst[f-1]
                current_rmse = rmse_lst[f]
                if current_rmse < previous_rmse:
                    self.alpha *= 1.1
                else:
                    self.alpha *= 0.9
                    if previous_rmse > 0 and ((previous_rmse - current_rmse) / previous_rmse > 0.1):
                        self.P = previous_P
                        self.Q = previous_Q
                        self.bu = previous_bu
                        self.bi = previous_bi
                        self.b = previous_b
            print(f"Iteration {f+1}/{self.iterations}, RMSE: {rmse:.4f}, alpha: {self.alpha:.6f}")  
                    
            
            
        self.rmse = rmse_lst
            
    def ind_predict(self, tup):
        """
            tup: One single entry, (user, movie)
            
            TODO: 10: Use P and Q to make prediction on single entry.
            
        """
        u,i = tup[0], tup[1]
        return self.b + self.bu[u] + self.bi[i] + np.dot(self.P[u], self.Q[i])

    
    def predict(self, X):
        """
            X: list of entries
            
            TODO: 11: Use ind_predict we create to make predicitons.
        """
        return [self.ind_predict(tup) for tup in X]
            
        
    

### If you are doing thing correctly (With tuning), you should get a Test RMSE below 1.

In [6]:
%%time
clf = Matrix_Factorization_with_bias(iterations =70, num_of_latent = 80, lam = 0.01)
clf.fit(train)

Iteration 1/70, RMSE: 1.0865, alpha: 0.000010
Iteration 2/70, RMSE: 1.0855, alpha: 0.000011
Iteration 3/70, RMSE: 1.0844, alpha: 0.000012
Iteration 4/70, RMSE: 1.0833, alpha: 0.000013
Iteration 5/70, RMSE: 1.0821, alpha: 0.000015
Iteration 6/70, RMSE: 1.0807, alpha: 0.000016
Iteration 7/70, RMSE: 1.0793, alpha: 0.000018
Iteration 8/70, RMSE: 1.0777, alpha: 0.000019
Iteration 9/70, RMSE: 1.0761, alpha: 0.000021
Iteration 10/70, RMSE: 1.0743, alpha: 0.000024
Iteration 11/70, RMSE: 1.0724, alpha: 0.000026
Iteration 12/70, RMSE: 1.0704, alpha: 0.000029
Iteration 13/70, RMSE: 1.0682, alpha: 0.000031
Iteration 14/70, RMSE: 1.0659, alpha: 0.000035
Iteration 15/70, RMSE: 1.0635, alpha: 0.000038
Iteration 16/70, RMSE: 1.0609, alpha: 0.000042
Iteration 17/70, RMSE: 1.0582, alpha: 0.000046
Iteration 18/70, RMSE: 1.0553, alpha: 0.000051
Iteration 19/70, RMSE: 1.0524, alpha: 0.000056
Iteration 20/70, RMSE: 1.0492, alpha: 0.000061
Iteration 21/70, RMSE: 1.0460, alpha: 0.000067
Iteration 22/70, RMSE:

In [7]:
error = 0
pred = clf.predict(test)
for i in range(len(test)):
    error += (test[i][2] - pred[i])**2

In [8]:
(error/ len(test))** (0.5)

np.float64(0.9590916371607499)